# PCA for spatial analysis


In this python file the seven steps of PCA for the spatial analysis is set up

1. Standardize all input variables to z-score
2. Select data for the indicators that shape the social vulnerability based on the correlation matrix
3. Perform the PCA with the standardized input values
4. Select the number of components to be further used
5. Rotate the PCA solution
6. Interpretate the resulting components on how they might influence vulnerability. Based on this, signs are assigned to the components. The output of the loadings is the determining factor for assigning the sign. The indicator with the highest loading in the component determines the sign. If this indicator is positively correlated with the social vulnerability, a positive sign will be assigned and vice versa.
7. The component scores are combined into a univariate score based on the predetermined weighting scheme


In [ ]:
#import the packags
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import scipy.stats as stats
from factor_analyzer import FactorAnalyzer

In [ ]:
# import the data set
df = pd.read_csv('social_vuln_goed.csv', delimiter = ';', decimal = ',')

## 00. Data preprocessing

In [ ]:
#drop all collumns that do not contain location name or indicators
df = df.drop([ 'bfa_popula', 'OID_1', 'Adm2_1', 'Pop_adm2',
       'OID_12', 'District_v', 'District_1', 'DS_pop', 'OID_12_13', 'Adm1_12', 'Count17',
       'Count18', 'Count19' ], axis = 1)

In [ ]:
df.nunique()

In [ ]:
#drop all rows that contain only 1 unique value
df = df.drop([ 'GII', 'Tuberculos', 'MalariaMor', 'MortRate__', 'HFA', 'Gov_Effect', 'CPI', 'Elektricit',
      'PhoneSubs', 'InternetUs', 'LiteractyR', 'HealthExp'], axis = 1)

In [ ]:
#change all rows to numeric
cols = df.columns.drop('Adm3')
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
#set the index to the communities
df = df.set_index('Adm3')

## 01. Standardization of data

In [ ]:
# select the columns that need to be standardized
cols = list(df.columns)
df[cols]

In [ ]:
# standardize with z-score
df_standardize = df.select_dtypes(include='number').apply(stats.zscore)
df_standardize.head()

## 02. Select the indicators based on the correlation matrix

In [ ]:
# create the correlation matrix
matrix = df_standardize.corr().round(2)

In [ ]:
# plot the figure
sns.set(rc = {'figure.figsize':(20, 20)})
sns_plot = sns.heatmap(matrix, annot=True, cmap = 'viridis')
plt.title('Correlation between the indicator datasets', fontname="Times New Roman", fontweight = 'bold', fontsize = '19')
plt.savefig('correlation_indicators.png')
plt.show()

Drop the variables on which other variables are based, based on pearson > 0.7

In [ ]:
df_standardize = df_standardize.drop(['PiN', 'Global_hum', 'total_ODA', 'ODA_of_GNI', 'GNI', 'Underweigh', 'MDPI', 'Electricit', 'IDPs_perc', 'Sanitation', 'Public_aid'], axis = 1)

In [ ]:
# rename the columns of the left over indicators
df_standardize.columns = ['Children < 5', 'Elderly > 60', 'No. Disabled', 'No. Conflict affected', 'No. Healthsites', 'Travel time [min]', 'Road Density [km/km\u00b2]',
                           'Undernourishment level 3-5 [%]', 'No. Malnutrition', 'Radio access [%]', 'Television access [%]', 'HDI', 'GINI', 'Prevalence HIV [%]', 'Water sources [%]', 
                          'Improved Sanitation [%]', 'Immunization measles [%]', 'No. Hazard affected',  'No. IDPs', 'No. Conflicts']

In [ ]:
# plot the correlation matrix again 

matrix = df_standardize.corr().round(2)
sns.set(rc = {'figure.figsize':(50, 35)})
sns_plot = sns.heatmap(matrix, annot=True, cmap = 'viridis', annot_kws={"fontsize":25})
plt.yticks(fontsize=30)
plt.xticks(fontsize=30)
plt.title('Correlation between the selected indicators', fontname="Times New Roman", fontweight = 'bold', fontsize = 40)
plt.savefig('correlation_indicators_pearsons.png')

plt.show()

## 03. Perform the PCA

In [ ]:
# use factor analyzer to calculate PCA with varimax rotation
fa = FactorAnalyzer(n_factors=20, method='principal', rotation="varimax")
fa.fit(df_standardize)
loadings_all = fa.loadings_.round(2)


In [ ]:
# plots original and common factor eigen values
fa.get_eigenvalues()

In [ ]:
#get variance
# for orignial vairance use fa.get_factor_variance[0]
# for proportional variance use fa.get_factor_variance[1]
# for cumulative variance use ga.get_factor_variance[2]

fa.get_factor_variance()


## 04. Select the number of components that is used for the analysis

Sort the data frame based on the amount of variance explained by each component

In [ ]:
#create a dataframe based on the loading, and name the column headers accordingly
df = pd.DataFrame(loadings_all, columns = ['PC1','PC2','PC3', 'PC4', 'PC5', 'PC6', 'PC7','PC8','PC9', 'PC10', 'PC11', 'PC12', 
                                           'PC13','PC14','PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20'])

#create list of variable names
indicator_names = df_standardize.columns.values.tolist()

#set list with indicator names as index PC df of eigen vectors
df['variables'] = indicator_names
df = df.set_index('variables')

#round the number on two decimals
df = df.round(2)

In [ ]:
# append the dataframe with the explained variance of each principal component
variance = fa.get_factor_variance()[1]

df.loc[len(df.index)] = variance

# sort the data frame based on the explained variance 
df_sorted = df.sort_values(by =20, axis=1,ascending=False)


In [ ]:
# add a row with the cumulative variance to the data frame
cumulative_variance = []
cum_variance = 0

for i in df_sorted.columns:
    cum_variance += df_sorted[i][20]
    cumulative_variance.append(cum_variance)

In [ ]:
# sort the variance lists
sorted_cumulative = df_sorted.loc[21]
sorted_variance = df_sorted.loc[20]

In [ ]:
plt.plot(sorted_cumulative, linewidth = 10)
plt.xlabel('Number of components', fontsize = 35, fontname="Times New Roman")
plt.ylabel('Cumulative explained variance',  fontsize = 35, fontname="Times New Roman")
plt.title('Cumulative explained variance, \n 15 PC are needed to explain 90% variance',  fontsize = 45, fontname="Times New Roman", fontweight = 'bold')
plt.yticks(fontsize=35)
plt.xticks(fontsize=35)
plt.axvline(x=14.2, color = 'grey', linestyle = '--', linewidth = 10)
plt.axhline(y=0.9, color='grey', linestyle='--', linewidth = 10)
plt.plot(14.2,.9, marker="o", color="red", markersize=20)
plt.savefig('90%varimax_2020_all_indi.png')
plt.show()

In [ ]:
PC_values =  list(range(1, 21))
plt.plot(PC_values,sorted_variance, 'ro-', linewidth=2)
plt.title('Scree Plot', fontsize = 45, fontname="Times New Roman", fontweight = 'bold' )
plt.xlabel('Principal Component', fontsize = 35, fontname="Times New Roman", fontweight = 'bold')
plt.ylabel('Proportion of Variance Explained', fontsize = 35, fontname="Times New Roman", fontweight = 'bold')
plt.yticks(fontsize=35)
plt.xticks(fontsize=35)
plt.savefig('Screeplot_2020_all_indi.png')
plt.show()

## 05. Rotate the PCA with the right number of components

In [ ]:
#use factor analyzer to calculate PCA with varimax rotation, from the loading matrix, select the number of components that is needed
loading_matrix = df_sorted[['PC5', 'PC1', 'PC12', 'PC16', 'PC2', 'PC6', 'PC4', 'PC15', 'PC7',
       'PC11', 'PC17', 'PC9', 'PC8', 'PC10', 'PC3']]
loading_matrix

Obtain the scores

In [ ]:
loading_matrix_scores = loading_matrix.drop([21, 20], axis = 0)
loading_matrix_scores

In [ ]:
scores = df_standardize.dot(loading_matrix_scores)
scores = scores.round(2)


## 06. Interpretate the resulting components on how they might influence vulnerability. 
Based on this, signs are assigned to the components. The output of the loadings is the determining factor for assigning the sign. The indicator with the highest loading in the component determines the sign. If this indicator is positively correlated with the social vulnerability, a positive sign will be assigned and vice versa

In [ ]:
def highlight(x):
    colors = 'background-color: lightgreen;'
    default = ''
    if type(x) in [float, int]:

        if x < -0.7:
            return colors
        elif x > 0.7:
            return colors
        else:
            return default
        
loading_matrix_scores.style.applymap(highlight)

In [ ]:
#set scores table to impact on vulnerbaility
scores['PC5'] = np.abs(scores['PC5'])
scores['PC1'] = np.abs(scores['PC1'])
scores['PC12'] = np.abs(scores['PC12'])*-1
scores['PC16'] = np.abs(scores['PC16'])*-1
scores['PC2'] = np.abs(scores['PC2'])*-1
scores['PC6'] = np.abs(scores['PC6'])
scores['PC4'] = np.abs(scores['PC4'])*-1
scores['PC15'] = np.abs(scores['PC15'])
scores['PC7'] = np.abs(scores['PC7'])*-1
scores['PC11'] = np.abs(scores['PC11'])
scores['PC17'] = np.abs(scores['PC17'])*-1
scores['PC9'] = np.abs(scores['PC9'])*-1
scores['PC8'] = np.abs(scores['PC8'])
scores['PC10'] = np.abs(scores['PC10'])*-1
scores['PC3'] = np.abs(scores['PC3'])

## 07. Calculate the scores for social vulnerability

In [ ]:
scores['vulnerability'] = scores.sum(axis = 1)


In [ ]:
scores.to_csv('vulnerability_cutter_2020_all_indi_varimax_try2_final.csv', decimal = ',') 

## 08. Interpretation of results

In [ ]:
# renamce the PC with the character they describe
plot = plot.rename(columns={'PC5': 'PC5 - IDPs', 'PC1': 'PC1 - Hazard','PC2': 'PC2 - HDI', 'PC12': 'PC12 - Travel time', 'PC16': 'PC16 - Television access', 'PC4': 'PC4 - Sanitation',
                            'PC15': 'PC15 - Conflict', 'PC7': 'PC7 - Road density ', 'PC11': "PC11 - Malnutrition", 'PC17' : 'PC17 - Water sources', 'PC9' : 'PC9 - Measle immuni', 
                            'PC8': 'PC8 - Children < 5', 'PC10': 'PC10 - Healthsites', 'PC6': 'PC6 - HIV', 'PC3': 'PC3 - Elderly'})

In [ ]:
plot.plot.barh(stacked=True, cmap = 'RdYlGn', figsize = (20,15))
plt.xlabel('Contribution to vulnerability score', fontsize = 35, fontname="Times New Roman")
plt.ylabel('Community',  fontsize = 35, fontname="Times New Roman")
plt.title('Composition of vulnerability score \n of ten most vulnerable communes',  fontsize =45, fontname="Times New Roman", fontweight = 'bold')
plt.yticks(fontsize=55)
plt.xticks(fontsize=55)
plt.legend(loc='best', prop={'size': 15})
plt.savefig('vulnerability_composition.png')
plt.show();

## 09. KMO

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_standardize)
kmo_model

## 10. Barlett